In [33]:
# Cell 1: Import Libraries

import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier as XGBC
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import KFold, cross_val_score as CVS, train_test_split as TTS
import lightgbm as lgb

import catboost as ctb


In [34]:


# 直接在读取CSV时只选择需要的列
# data = pd.read_csv('merge.csv')
data = pd.read_csv('mergedata/merged_data.csv')
# data = pd.read_csv('Android_Malware.csv')
# # data = pd.read_csv('datae.csv', usecols=core_features)

# # 显示数据信息
data.head()

,Flow_ID,Source_IP,Source_Port,Destination_IP,Destination_Port,Protocol,Timestamp,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,1.234.63.69-10.42.0.211-80-51452-6,10.42.0.211,51452,1.234.63.69,80,6,22/06/2017 09:24:06,842,2,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,1.234.63.69-10.42.0.211-80-48934-6,10.42.0.211,48934,1.234.63.69,80,6,22/06/2017 09:24:06,1093891,7,12,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,1.234.63.69-10.42.0.211-80-55124-6,10.42.0.211,55124,1.234.63.69,80,6,22/06/2017 09:24:07,218208,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,1.234.63.69-10.42.0.211-80-59233-6,10.42.0.211,59233,1.234.63.69,80,6,22/06/2017 09:24:07,218249,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,1.234.63.69-10.42.0.211-80-49392-6,10.42.0.211,49392,1.234.63.69,80,6,22/06/2017 09:24:07,218245,1,1,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [35]:
# 获取数据集的特征信息
# print("数据集特征名称:")
# print(data.columns)

# print("\n数据集基本信息:")
# print(data.info())

# print("\n数据集统计描述:")
# print(data.describe())


In [36]:
# data.head()

In [37]:
# Cell 4: 数据类型转换

# Convert all columns (except 'Label') to numeric types
for col in data.columns:
    if col != 'Label':  # 修改为带空格的列名
        try:
            series = pd.to_numeric(data[col], errors='coerce')
            series = series.replace([np.inf, -np.inf], np.nan)
            series = series.fillna(0).astype(int)
            data[col] = series
        except Exception as e:
            print(f"Error converting column {col}: {e}")

print("Data types after conversion:")
# print(data.dtypes)
# # Cell 4: 数据类型转换

# # Convert all columns (except 'Label') to numeric types
# for col in dataceshi.columns:
#     if col != ' Label':  # 修改为带空格的列名
#         try:
#             series = pd.to_numeric(dataceshi[col], errors='coerce')
#             series = series.replace([np.inf, -np.inf], np.nan)
#             series = series.fillna(0).astype(int)
#             dataceshi[col] = series
#         except Exception as e:
#             print(f"Error converting column {col}: {e}")

# print("Data types after conversion:")
# # print(dataceshi.dtypes)

Data types after conversion:


In [38]:
# Cell 5: 标签映射

# 确保标签值为整数类型
data['Label'] = data['Label'].map(lambda x: 0 if x == 'BENIGN' else 1).astype(int)

print("Label column after mapping:")
print(data['Label'].value_counts())
# # data.info()
# dataceshi[' Label'] = dataceshi[' Label'].map(lambda x: 0 if x == 'BENIGN' else 1).astype(int)

# print(" Label column after mapping:")
# print(dataceshi[' Label'].value_counts())
# # data.info()

Label column after mapping:
Label
0    99615
1    32701
Name: count, dtype: int64


In [39]:
# Cell 6: 清除缺失值

data = data.dropna()
print("After dropping missing values, data shape:", data.shape)
# dataceshi = dataceshi.dropna()
# print("After dropping missing values, data shape:", dataceshi.shape)


After dropping missing values, data shape: (132316, 84)


In [40]:
# data.head(5)

In [41]:
# Cell 7: 数据分割

X = data.drop(['Label'], axis=1)  # 使用带空格的列名
y = data['Label']  # 使用带空格的列名

X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.01, random_state=42, stratify=y
)

print("Training set shape:", X_train.shape, Y_train.shape)
print("Test set shape:", X_test.shape, Y_test.shape)

Training set shape: (130992, 83) (130992,)
Test set shape: (1324, 83) (1324,)


In [42]:
# X_train = data.drop([' Label'], axis=1)  # 使用带空格的列名
# Y_train = data[' Label']  # 使用带空格的列名
# X_test = dataceshi.drop([' Label'], axis=1)  # 使用带空格的列名
# Y_test = dataceshi[' Label']  # 使用带空格的列名

In [43]:
for dataset in [X_train, X_test, Y_train, Y_test]:
    dataset.index = range(dataset.shape[0])

In [44]:
# clf= xgb.XGBClassifier(n_estimator='100',use_label_encoder=False,silent=False).fit(X_train,Y_train) #训练

In [45]:
# clf.score(X_test,Y_test) #测试

In [46]:

# 对于回归模型(regressor)，score方法返回的是R²决定系数（取值范围负无穷到1），而非准确率。具体取决于模型类型：

# 分类器(Classifier).score() → 返回准确率
# 回归器(Regressor).score() → 返回R²分数

In [47]:
# clf.feature_importances_

In [48]:
# cross_val_score(clf,X_train,Y_train,cv=3).mean() #交叉验证

In [49]:
# Cell 8: 类别权重计算

neg_count, pos_count = np.bincount(Y_train)  # y_train已经是整数类型,不需要再转换
scale_pos_weight = neg_count / pos_count
print("scale_pos_weight:", scale_pos_weight)

scale_pos_weight: 3.046209921541978


In [50]:
import catboost as ctb
from sklearn.metrics import accuracy_score

# 定义CatBoost回归模型，启用GPU
cat_clf = ctb.CatBoostClassifier(
                                # learning_rate=0.1,
                                iterations=2000,
                                depth=8,
                                random_seed=32,
                                task_type='GPU',  # 启用GPU训练
                                devices='0'       # 指定GPU设备
                                )

# 训练模型，仅使用测试集作为eval_set
cat_clf.fit(X_train, Y_train,
            eval_set=[(X_test, Y_test)],  # 只传入一个评估集
            verbose=100)                  # 每100次迭代输出信息

# 进行预测并计算分类准确率
cat_predict = cat_clf.predict(X_test)
# 计算准确率
accuracy = accuracy_score(Y_test, cat_predict)
print(f"模型准确率: {accuracy:.4f}")
cat_predict = cat_clf.predict(X_test)
print("预测结果（前100个）：", cat_predict[:100])

0:	learn: 0.6495489	test: 0.6528917	best: 0.6528917 (0)	total: 29.5ms	remaining: 29.5s
100:	learn: 0.4082982	test: 0.4279592	best: 0.4279592 (100)	total: 1.19s	remaining: 10.6s
200:	learn: 0.3730292	test: 0.3923306	best: 0.3923306 (200)	total: 2.21s	remaining: 8.79s
300:	learn: 0.3509910	test: 0.3738592	best: 0.3738592 (300)	total: 3.23s	remaining: 7.49s
400:	learn: 0.3333371	test: 0.3600876	best: 0.3600876 (400)	total: 4.23s	remaining: 6.32s
500:	learn: 0.3201938	test: 0.3514632	best: 0.3514632 (500)	total: 5.23s	remaining: 5.21s
600:	learn: 0.3083251	test: 0.3420461	best: 0.3420461 (600)	total: 6.22s	remaining: 4.13s
700:	learn: 0.2976246	test: 0.3357202	best: 0.3357202 (700)	total: 7.21s	remaining: 3.08s
800:	learn: 0.2884090	test: 0.3314748	best: 0.3314505 (798)	total: 8.21s	remaining: 2.04s
900:	learn: 0.2797562	test: 0.3266457	best: 0.3265578 (895)	total: 9.19s	remaining: 1.01s
999:	learn: 0.2722596	test: 0.3229504	best: 0.3228491 (991)	total: 10.2s	remaining: 0us
bestTest = 0.32

In [51]:
# Bestparameters= {'subsample': np.float64(0.8999999999999999), 'n_estimators': np.int64(450), 'max_depth': np.int64(9), 'learning_rate': np.float64(0.21544346900318823), 'colsample_bytree': np.float64(0.7999999999999999)}

# # 初始化XGBoost分类器，使用GPU（直接设置固定参数）
# clf = xgb.XGBClassifier(
#     objective='binary:logistic',        # 二元分类，输出概率
#     booster='gbtree',                  # 基于树的增强器
#     device='cuda',                     # 启用GPU加速
#     tree_method='gpu_hist',            # 使用GPU直方图算法
#     subsample=Bestparameters['subsample'], # 子样本比例
#     n_estimators=Bestparameters['n_estimators'], # 树的数量
#     max_depth=Bestparameters['max_depth'], # 树的最大深度
#     learning_rate=Bestparameters['learning_rate'], # 学习率
#     colsample_bytree=Bestparameters['colsample_bytree'], # 列采样比例
#     random_state=42,                   # 随机种子
#     verbosity=1,                       # 显示训练过程
#                # 二元分类
#     # 确保预测也使用GPU（可选）)   
#     scale_pos_weight=scale_pos_weight, # 平衡正负类权重
#     predictor='gpu_predictor'          # 使用GPU预测
# )

# clf.fit(X_train, Y_train) #训练
# clf.score(X_test, Y_test) #测试


In [52]:
clf = xgb.XGBClassifier(
      
    # alpha=0.05,                     # L1 正则化
    # learning_rate=0.2,              # 学习率
                    # L2 正则化
    # gamma=0.5,                     # 最小损失减少量
    max_depth=9,                   # 树深度
    # min_child_weight=5,            # 子节点最小权重
    # subsample=0.7,                 # 样本采样比例
    # colsample_bytree=0.7,          # 特征采样比例
    device='cuda',        # 使用 GPU 加速
    n_estimators=1000,              # 树的数量
    random_state=30,              # 随机种子
    verbosity=1,                   # 显示训练过程
    scale_pos_weight=scale_pos_weight, # 平衡正负类权重
    objective='binary:logistic',    # 二元分类
      # 明确指定使用GPU加速
    
)
    

In [53]:
clf.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=30, ...)

In [54]:
clf.score(X_test, Y_test) #测试

0.8897280966767371

In [55]:
# 导入必要的库
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# 假设已有训练好的XGBoost和CatBoost模型：clf和cat_clf
# 以及训练集X_train, Y_train和测试集X_test, Y_test

# 获取两个基模型在训练集上的预测概率
xgb_train_pred_proba = clf.predict_proba(X_train)
cat_train_pred_proba = cat_clf.predict_proba(X_train)

# 将训练集的预测概率拼接作为元模型的训练特征
meta_train_features = np.column_stack((xgb_train_pred_proba, cat_train_pred_proba))

# 获取两个基模型在测试集上的预测概率
xgb_test_pred_proba = clf.predict_proba(X_test)
cat_test_pred_proba = cat_clf.predict_proba(X_test)

# 将测试集的预测概率拼接作为元模型的测试特征
meta_test_features = np.column_stack((xgb_test_pred_proba, cat_test_pred_proba))

# 定义并训练元模型（逻辑回归，适合二元分类）
meta_model = LogisticRegression()
meta_model.fit(meta_train_features, Y_train)

# 使用元模型对测试集特征进行预测
final_predictions = meta_model.predict(meta_test_features)

# 计算堆叠法集成后的准确率
stacking_accuracy = accuracy_score(Y_test, final_predictions)
print(f'堆叠法集成后的准确率: {stacking_accuracy:.4f}')

# 计算并对比单个模型的准确率
xgb_predictions = clf.predict(X_test)
cat_predictions = cat_clf.predict(X_test)
xgb_accuracy = accuracy_score(Y_test, xgb_predictions)
cat_accuracy = accuracy_score(Y_test, cat_predictions)
print(f'XGBoost模型准确率: {xgb_accuracy:.4f}')
print(f'CatBoost模型准确率: {cat_accuracy:.4f}')

堆叠法集成后的准确率: 0.8882
XGBoost模型准确率: 0.8897
CatBoost模型准确率: 0.8595


In [56]:
clf.save_model('model/xgb_model.json')

In [57]:
cat_clf.save_model('model/cat_model.cbm')

In [58]:
import pickle
with open('model/meta_model.pkl', 'wb') as f:
    pickle.dump(meta_model, f)

In [59]:
clf.score(X_train,Y_train) #测试

0.9960379259802126

In [60]:
# clf.feature_importances_

In [61]:
# # Cell 10: 特征重要性分析

# # 获取特征重要性分数
# feature_importance = clf.feature_importances_

# # 创建特征名称和重要性分数的DataFrame
# feature_importance_df = pd.DataFrame({
#     'feature': X.columns,
#     'importance': feature_importance
# })

# # 按重要性降序排序
# feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)

# # 绘制特征重要性条形图
# plt.figure(figsize=(20, 6))
# plt.bar(range(len(feature_importance_df)), feature_importance_df['importance'])
# plt.xticks(range(len(feature_importance_df)), feature_importance_df['feature'], rotation=45, ha='right')
# plt.xlabel('Features')
# plt.ylabel('Importance')
# plt.title('Feature Importance')
# plt.tight_layout()
# plt.show()

# # 打印前10个最重要的特征
# print('\nTop 10 Most Important Features:')
# print(feature_importance_df.head(40))

In [62]:
# low_importance_features

In [63]:
# low_importance_feature_list = low_importance_features['feature'].tolist()
# low_importance_feature_list

In [64]:
# # Cell 11: 模型评估

# try:
#     y_pred = xgb.predict(X_test)
#     y_pred_proba = xgb.predict_proba(X_test)[:, 1]

#     print("Classification Report:")
#     print(classification_report(Y_test, y_pred))

#     roc_auc = roc_auc_score(Y_test, y_pred_proba)
#     print("Test ROC-AUC score:", roc_auc)
# except Exception as e:
#     print("Error during evaluation:", e)